# 基本框架学习

## urllib

request：最基本的HTTP请求模块，可以用来模拟发送请求  
error：异常处理模块。可以捕获这些异常，然后进行重试或其他操作  
parse：一个工具模块，提供了许多URL处理方法，如拆分、解析、合并  
robotparse：识别robots.txt文件  

#### urlopen（）

In [1]:
import urllib.request
response = urllib.request.urlopen('https://www.python.org/')
print(response.read().decode('utf-8'))

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [2]:
print(type(response))

<class 'http.client.HTTPResponse'>


  可以发现，它是一个HTTPResponse类型的对象，主要包含read（）、readinto（）、getheader（name）、getheaders（）、fileno（）等方法，以及msg、version、reason、debuglevel、closed等属性  
  例如，调用read（）方法可以得到返回的网页内容，调用status属性可以得到返回结果的状态码  

In [6]:
import urllib.request

response = urllib.request.urlopen('https://www.python.org')
print(response.status)
print(response.getheaders())
print(response.getheader('Server'))

200
[('Connection', 'close'), ('Content-Length', '50875'), ('Server', 'nginx'), ('Content-Type', 'text/html; charset=utf-8'), ('X-Frame-Options', 'DENY'), ('Via', '1.1 vegur, 1.1 varnish, 1.1 varnish'), ('Accept-Ranges', 'bytes'), ('Date', 'Wed, 04 May 2022 13:40:22 GMT'), ('Age', '295'), ('X-Served-By', 'cache-iad-kcgs7200049-IAD, cache-tyo11925-TYO'), ('X-Cache', 'HIT, HIT'), ('X-Cache-Hits', '6, 231'), ('X-Timer', 'S1651671622.485811,VS0,VE0'), ('Vary', 'Cookie'), ('Strict-Transport-Security', 'max-age=63072000; includeSubDomains')]
nginx


前两个输出了状态码和头信息，最后一个通过调用getheader（）方法并传递一个参数Server获取了相应头中的Server的值，结果是Nginx意为服务器是用Nginx搭建的

In [8]:
#urllib.request.urlopen(url,data=None,[timeout,]*,cafile,capath=None.cadefault=False,context=None)

data参数

In [10]:
import urllib.parse

data = bytes(urllib.parse.urlencode({'word':'hello'}),encoding='utf-8')
response = urllib.request.urlopen('http://httpbin.org/post',data=data)
print(response.read())

b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "word": "hello"\n  }, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Content-Length": "10", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "User-Agent": "Python-urllib/3.9", \n    "X-Amzn-Trace-Id": "Root=1-62728593-6c5b1d5b26718d651c9a6772"\n  }, \n  "json": null, \n  "origin": "111.181.30.190", \n  "url": "http://httpbin.org/post"\n}\n'


传递了一个参数word，值为hello。它需要被转码成bytes（字节流）类型。其中转字节流采用了bytes（）方法，第一个参数需要str类型，故用urllib.parse模块里的urlencode（）方法来将参数字典转化为字符串。

timeout参数

In [14]:
response = urllib.request.urlopen('https://httpbin.org/get',timeout=1)
print(response.read())

b'{\n  "args": {}, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Host": "httpbin.org", \n    "User-Agent": "Python-urllib/3.9", \n    "X-Amzn-Trace-Id": "Root=1-62728757-45a082ec552ced68513f297a"\n  }, \n  "origin": "111.181.30.190", \n  "url": "https://httpbin.org/get"\n}\n'


对于超时的，可以通过设置这个超时时间来控制一个网页如果长时间为响应，就跳过它的抓取。这可以利用try except语句来实现

In [17]:
import socket
import urllib.request
import urllib.error

try:
    response = urllib.request.urlopen('http://httpbin.org/get',timeout=0.1)
except urllib.error.URLError as e:
    if isinstance(e.reason,socket.timeout):
        print('TIME OUT')

TIME OUT


#### Request

In [18]:
import urllib.request
request = urllib.request.Request('https://python.org')
response = urllib.request.urlopen(request)
print(response.read().decode('utf-8'))

<!doctype html>
<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->
<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->
<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->
<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->

<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">
    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jqueryui/1.12.1/jquery-ui.min.js">

    <meta name="application-name" content="Python.org">
    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">
    <meta name="apple-mobile-web-app-title" content="Python.org">
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">

    <meta name="

class urllib.request.Request(url,data=None,headers={},origin_req_host=None,unverifiable=False,method=None)  
1.url参数必传  
2.data如果要传，必须传bytes类型的。如果为字典，可以先用urllib.parse模块里的urlencode（）编码  
3.headers是一个字典，它就是请求头，我们可以在构造请求时通过headers参数直接构造，也可以通过调用请求实例的add_header()方法添加  
添加请求头时最常用的用法就是通过参数修改User-Agent来伪装浏览器，默认的User-Agent时Python-urllib，可以通过修改它来伪装成浏览器 
4.origin_req_host指的是请求方的host名称或者IP地址  
5.unverifiable表示这个请求是否无法验证的。例如，我们请求一个HTML文档中的图片，但是我们没有自动抓取图像的权限，这时unverifiable的值为True  
5.method来指示方法，如GET、POST和PUT  

In [20]:
from urllib import request,parse

url = 'http://httpbin.org/post'
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32',
    'Host':'httpbin.org'
}

dict = {
    'name':'Germay'
}

data = bytes(parse.urlencode(dict),encoding='utf-8')
req = request.Request(url=url,data=data,headers=headers,method='POST')
response = request.urlopen(req)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name": "Germay"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "11", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32", 
    "X-Amzn-Trace-Id": "Root=1-62728e9d-61ff2b8552044816161ec669"
  }, 
  "json": null, 
  "origin": "111.181.30.190", 
  "url": "http://httpbin.org/post"
}



观察结果可以发现，成功设置了data、headers和method

另外，headers也可以用add_header（）方法来添加：  
req = request.Request(url=url,data=data,method='POST')  
req.add_header( 'User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32')

#### 高级用法

更强大的工具Handler。处理器，几乎可以做到HTTP请求中所有的事情

##### BaseHandler

它是所有其他Handler的父类，它提供了最基本的方法  
HTTPDefaultErrorHandler :用于处理HTTP相应错误，错误都会抛出HTTPError类型的异常。  
HTTPRedirectHandler ：用于处理重定向  
HTTPCookiesProcessor ：用于设置代理，默认代理为空  
HTTPPasswordMgr ：用于密码管理，它维护了用户名和密码的表  
ProxyHandler ：用于设置代理，默认代理为空  g
HTTPBasicAuthHandler ：用于密码管理，如果一个连接打开时需要认证，那么可以用它解决认证问题  

另一个比较重要的类就是OpenerDirector，我们可以称为Opener。Opener可以使用open（）方法，返回的类型和urlopen（）如初一辙。

In [2]:
#验证
from urllib.request import HTTPPasswordMgrWithDefaultRealm,HTTPBasicAuthHandler,build_opener
from urllib.error import URLError
import urllib.request

username = 'username'
password = 'password'
url = 'http://localhost:5000/'

p = HTTPPasswordMgrWithDefaultRealm()
p_add_password(None,url,username,password)
auth_handler = HTTPBasicAuthHandler(p)
opener = build_opener(auth_handler)

try:
    result = opener.open(url)
    html = result.read().decode('utf-8')
    print(html)
except URLError as e:
    print(e.reason)

NameError: name 'p_add_password' is not defined

##### 代理

In [5]:
from urllib.error import URLError
from urllib.request import ProxyHandler,build_opener

proxy_handler = ProxyHandler({
    'http':'http://127.0.0.1:9743',
    'https':'https://127.0.0.1:9743'
})
opener = build_opener(proxy_handler)
try:
    response = opener.open('https://www.baidu.com')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

[WinError 10061] 由于目标计算机积极拒绝，无法连接。


这里在本地搭建了一个代理，它运行在9743端口上

这里使用了ProxyHandler，其参数是一个字典，键名是协议类型（比如HTTP和HTPPS等），键值是代理链接，可以添加多个代理。  
然后利用这个Handler及build_opener（）方法构造一个Opener，之后发送请求即可

#### Cookies

In [16]:
import http.cookiejar,urllib.request

cookie = http.cookiejar.CookieJar()
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
response = opener.open('http://www.baidu.com')
for item in cookie:
    print(item.name+"="+item.value)

BAIDUID=A62FC0B3B42DFFF4FBAD0D6EC6555F94:FG=1
BIDUPSID=A62FC0B3B42DFFF48934C2156F6B2AA4
H_PS_PSSID=36309_31254_34812_35912_36166_34584_35979_36075_35863_36232_26350_36302_36314_36061
PSTM=1651763831
BDSVRTM=0
BD_HOME=1


可以看到，给出了Cookie的名称和值。  
不过也可以输出成文件格式

In [20]:
filename = 'cookies.txt'
cookie = http.cookiejar.MozillaCookieJar(filename)
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
response = opener.open('http://www.baidu.com')
cookie.save(ignore_discard=True,ignore_expires=True)
print(cookies.txt)

NameError: name 'cookies' is not defined

#### 异常处理

#### URLError

URLError类来自urllib库的error模块，它继承自OSError类，是error异常模块的基类，由request模块生的异常都可以通过捕获这个类来处理|

In [24]:
from urllib import request,error
try:
    response = request.urlopen('https://cuiqingcai.com/index.htm')
except error.URLError as e:
    print(e.reason)

Not Found


#### HTTPError

它是URLError的子类，专门用来处理HTTP请求错误，比如认证请求失败等  
1.code:返回HTTP状态码  
2.reason：返回错误的原因  
3.headers：返回请求头

In [27]:
from urllib import request,error

try:
    response = request.urlopen('https://cuiqingcai.com/index.htm')
except error.URLError as e:
    print(e.reason,e.code,e.headers,end='\n')

Not Found 404 Server: GitHub.com
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
ETag: "6272da93-247b"
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; img-src data:; connect-src 'self'
x-proxy-cache: MISS
X-GitHub-Request-Id: 7754:1ABA:16F737:1941D8:6273EEBB
Accept-Ranges: bytes
Date: Thu, 05 May 2022 15:42:04 GMT
Via: 1.1 varnish
Age: 401
X-Served-By: cache-hkg17921-HKG
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1651765325.979535,VS0,VE1
Vary: Accept-Encoding
X-Fastly-Request-ID: dabdc0181208906d4c679059e0ee143ade2f663d
X-Cache-Lookup: Cache Miss
X-Cache-Lookup: Cache Miss
X-Cache-Lookup: Cache Miss
Content-Length: 9339
X-NWS-LOG-UUID: 17501831352823260970
Connection: close
X-Cache-Lookup: Cache Miss




由于URLError是HTTPError的父类，所以可以先选择捕获子类的错误，再去捕获父类的错误，所以上述代码更好的写法如下

In [31]:
from urllib import request,error

try:
    response = request.urlopen('https://cuiqingcai.com/index.htm')
except error.HTTPError as e:
    print(e.reason,e.code,e.headers,end='\n')
except error.URLError as e:
    print(e.reason)
else :
    print('Request Successfully')

Not Found 404 Server: GitHub.com
Content-Type: text/html; charset=utf-8
Access-Control-Allow-Origin: *
ETag: "6272da93-247b"
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; img-src data:; connect-src 'self'
x-proxy-cache: MISS
X-GitHub-Request-Id: 7754:1ABA:16F737:1941D8:6273EEBB
Accept-Ranges: bytes
Date: Thu, 05 May 2022 15:48:45 GMT
Via: 1.1 varnish
Age: 801
X-Served-By: cache-hkg17935-HKG
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1651765725.122117,VS0,VE1
Vary: Accept-Encoding
X-Fastly-Request-ID: 2cb2260270a00f4595bf9916a46ce7ac277d11f0
X-Cache-Lookup: Cache Miss
X-Cache-Lookup: Cache Miss
Content-Length: 9339
X-NWS-LOG-UUID: 4660493620829011409
Connection: close
X-Cache-Lookup: Cache Miss




#### 解析链接

urllib库里还提供了parse模块，它定义了处理RUL的标准接口，实现URL各部分的抽取、合并以及链接转换。  
它支持如下的URL处理：  
file,ftp,gopher,hdl,http,https,imap,mailto,mms,news,nntp,prospero,rsync,rtsp,rtspu,sftp,sip,sips,snews,svn,svn+ssh,telnet和wais

##### urlparse()

In [32]:
from urllib.parse import urlparse

result = urlparse('https://www.baidu.com/index.html;user?id=5#comment')
print(type(result),result)

<class 'urllib.parse.ParseResult'> ParseResult(scheme='https', netloc='www.baidu.com', path='/index.html', params='user', query='id=5', fragment='comment')


可以看到，urlparse()方法将其拆分成了6个部分。大体观察可以发现，解析时由特定的分隔符，比如：//前面是scheme,代表协议；netloc即为域名；后面是path。即访问路径；前面是params，代表参数；问号？后面是查询条件query，一般用作GET类型的URL；井号后面是锚点，用于直接定位页面内部的下拉位置

一个标准的链接格式：  
scheme://netloc/path;params?query#fragment

一个标准的URL都会符合这个规则，利用urlparse()方法可以将它拆分开来。  
urllib.parse.urlparse(urlstring,scheme='',allow_fragments=True)  
1.urlstring:待解析的URL  
2.scheme：它是默认的协议（比如http或https等） 
3.allow_fragments:即是否忽略frament。如果被设置为False，fragment部分就会被忽略，它会被解析成为path、parameters或者query的一部分

In [2]:
from urllib.parse import urlparse

result = urlparse('www.baidu.com/index,html;user!id=5#comment',scheme='htpps')
print(result)

ParseResult(scheme='htpps', netloc='', path='www.baidu.com/index,html;user!id=5', params='', query='', fragment='comment')


可以看到，URL中没有scheme的内容，如果加上scheme就会有该内容了  

In [4]:
result = urlparse('http://www.baidu.com/index,html;user!id=5#comment',scheme='https')
print(result)

ParseResult(scheme='http', netloc='www.baidu.com', path='/index,html', params='user!id=5', query='', fragment='comment')


可以看到，就算scheme参数是https，如果URL中也是http，其结果仍为http

In [11]:
from urllib.parse import urlparse

result = urlparse('http://www.baidu.com/index.html;user!id=5#comment',allow_fragments=False)
print(result)

ParseResult(scheme='http', netloc='www.baidu.com', path='/index.html', params='user!id=5#comment', query='', fragment='')


假设URL中不含params和query

In [10]:
result = urlparse('http://www.baidu.com/index.html#comment',allow_fragments=False)
print(result)

ParseResult(scheme='http', netloc='www.baidu.com', path='/index.html#comment', params='', query='', fragment='')


返回结果ParseResult实际上是一个元组，可以用索引顺序来获取，也可以用属性名来获取

In [13]:
from urllib.parse import urlparse

result = urlparse('http://www.baidu.com/index.html#comment',allow_fragments=False)
print(result.scheme,result[0],result.netloc,result[1])

http http www.baidu.com www.baidu.com


除了urlparse（）还有urlunparse（），它接受的参数是一个可迭代的对象，但是长度必为6，否则会出现参数数量不足或者过多的问题

In [15]:
from urllib.parse import urlunparse

data = ['http','www.baidu.com','index.html','user','a=6','comment']
print(urlunparse(data))

http://www.baidu.com/index.html;user?a=6#comment


##### urlsplit

这个方法和urlparse()方法非常类似，只不过它不再单独解析params这一部分，只返回5个结果。

In [3]:
from urllib.parse import urlsplit
result = urlsplit('https://www.baidu.com/index.html;user?id=5#comment')
print(result)

SplitResult(scheme='https', netloc='www.baidu.com', path='/index.html;user', query='id=5', fragment='comment')


返回结果是SplitResult，它其实是一个元组类型，既可以用属性获取值，也可以用索引来获取

##### urlunsplit

与urlunsplit（）类似，它也是将链接各个部分组合完成完整链接的方法，传入的参数也是一个可迭代的对象，例如列表、元组等，唯一的区别是长度必须为5.

In [5]:
from urllib.parse import urlunsplit

data = ['http','www.baidu.com','index.html','a=6','comment']
print(urlunsplit(data))

http://www.baidu.com/index.html?a=6#comment


##### urljoin()

可以提供一个base_url（基础链接）作为第一个参数，将新的链接作为第二个参数，该方法会分析base_url的scheme、netloc和path这3个内容并对新连接缺失部分的进行补充

In [6]:
from urllib.prase import urljoin

ModuleNotFoundError: No module named 'urllib.prase'

## requests

urllib库中的urlopen（）方法实际上以GET方式请求网页，而requests中相应的方法就是get()方法。

In [7]:
import requests

r = requests.get('https://www.baidu.com/')
print(type(r))
print(r.status_code)
print(type(r.text))
print(r.text)
print(r.cookies)

<class 'requests.models.Response'>
200
<class 'str'>
<!DOCTYPE html>
<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=https://ss1.bdstatic.com/5eN1bjq8AAUYm2zgoY3K/r/www/cache/bdorz/baidu.min.css><title>ç¾åº¦ä¸ä¸ï¼ä½ å°±ç¥é</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt value maxlengt

该处使用了ger（）方法实现与urlopen()相同的操作，得到一个Response()对象，然后分别输出了Response的类型、状态码、响应体的类类型、内容以及Cookies。  
可以发现，返回类型是requests.models.Response，响应体的类型是字符串str,Cookies的类型是RequestsCookieJar

### GET

添加额外的信息

In [8]:
import requests

data = {
    'name':'germey',
    'age':22
}
r = requests.get("http://httpbin.org/get",params=data)
print(r.text)

{
  "args": {
    "age": "22", 
    "name": "germey"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.26.0", 
    "X-Amzn-Trace-Id": "Root=1-62791a7c-1111008f4688625d33388266"
  }, 
  "origin": "171.43.245.215", 
  "url": "http://httpbin.org/get?name=germey&age=22"
}



通过结果可知，请求的链接自动被构造成了：http://httpbin.org/get?age=22&name=germey  
另外，网页的返回结果实际上str类型，但是它很特殊，是JSON格式。所以，想要直接返回结果，得到一个字典格式的话，可以直接调用json（）方法。

In [9]:
import requests

r = requests.get("http://httpbin.org/get")
print(type(r.text))
print(r.json())
print(type(r.json()))

<class 'str'>
{'args': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.26.0', 'X-Amzn-Trace-Id': 'Root=1-62791b6b-07f7c0d83cccd87532edc62e'}, 'origin': '171.43.245.215', 'url': 'http://httpbin.org/get'}
<class 'dict'>


#### 抓取网页

In [2]:
import requests
import re

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'
}
r = requests.get("https://www.zhihu.com/explore",headers=headers)
pattern = re.compile('explore-feed.*?question_link.*?>(.*?)</a>',re.S)
titles = re.findall(pattern,r.text)
print(titles)

[]


#### 抓取二进制数据

In [15]:
import requests

r = requests.get("https://github.com/favicon.ico")
print(r.text)
print(r.content)

SyntaxError: invalid syntax (Temp/ipykernel_11204/2626613227.py, line 5)

前者是r.text的结果，另一个是content  
后者有一个b，代表是bytes的数据。由于图片是二进制数据，所以前者在打印时转化为str类型，也就是图片直接转化为字符串。

In [17]:
import requests

r = requests.get("https://github.com/favicon.ico")
with open('favicon.ico','wb') as f:
    f.write(r.content)

ConnectionError: HTTPSConnectionPool(host='github.com', port=443): Max retries exceeded with url: /favicon.ico (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000203E3F79EB0>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

这里使用了open（）方法，它的第一个参数是文件名，第二个参数代表以二进制写的形式打开，可以向文件里写入二进制数据

#### 添加headers

与urllib.request一样，也可以通过headers参数来传递头信息

### POST

In [19]:
import requests

data = {'name':'germey','age':'22'}
r = requests.post("http://httpbin.org/post",data=data)
print(r.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "age": "22", 
    "name": "germey"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Content-Length": "18", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.26.0", 
    "X-Amzn-Trace-Id": "Root=1-6279243a-7e9f62f00badf1b44303c248"
  }, 
  "json": null, 
  "origin": "171.43.245.215", 
  "url": "http://httpbin.org/post"
}



### 响应

在上面的实例中，我们使用了text和content获取了相应的内容。此外，还有很多属性和方法可以用来获取其他信息，比如状态码、响应头、Cookies等

In [22]:
import requests

r = requests.get('http://www.jianshu.com')
print(type(r.status_code),r.status_code)
print(type(r.headers),r.headers)
print(type(r.cookies),r.cookies)
print(type(r.url),r.url)
print(type(r.history),r.history)

<class 'int'> 403
<class 'requests.structures.CaseInsensitiveDict'> {'Server': 'Tengine', 'Date': 'Mon, 09 May 2022 14:33:52 GMT', 'Content-Type': 'text/html', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Content-Encoding': 'gzip'}
<class 'requests.cookies.RequestsCookieJar'> <RequestsCookieJar[]>
<class 'str'> https://www.jianshu.com/
<class 'list'> [<Response [301]>]


In [23]:
import requests

r = requests.get('http://www.jianshu.com')
exit() if not r.status_code == requests.codes.ok else print('Request Successfully')

In [1]:
import requests

files = {'file':open('favicon.ico','rb')}
r = requests.post("http://httpbin.org/post",files=files)
print(r.text)

FileNotFoundError: [Errno 2] No such file or directory: 'favicon.ico'

### 高级用法

### 文件上传

#### Cookies

In [4]:
import requests

r = requests.get("https://www.baidu.com")
print(r.cookies)
for key,value in r.cookies.items():
    print(key + '=' + value)

<RequestsCookieJar[<Cookie BDORZ=27315 for .baidu.com/>]>
BDORZ=27315


这里我们首先调用cookies属性既可成功得到Cookies，可以发现它是RequestCookieJar类型。然后用items()方法将其转化为元组组成的列表

In [16]:
import requests

headers = {
    'Cookie':'_zap=914c7061-bc67-40d8-9d2f-307d9203368a; d_c0="APCfpUiOjROPTny1k41UpIDGLfgRghMrJsg=|1628697163"; _xsrf=ZFeEdqQ2pLrCK55wCH5EZVkKZkOkpXRf; __snaker__id=jxZ3erivx0VO75TL; _9755xjdesxxd_=32; YD00517437729195:WM_TID=eHSkDGk8jLlAQUFAQAJ79YQGeWeQef1b; q_c1=45705d6745e14ca2aa7e8db26508061c|1645972962000|1645972962000; captcha_session_v2=2|1:0|10:1652104512|18:captcha_session_v2|88:d2xKZUROR3pGb01nTEQrT2gvNytrR2k1Uk5MYVo2aDVlUUUzeFVESjFlazBkUEVwa3RrYlJqaS85TkJYRHFjNw==|5741ff6f88b2c3bd13c26dce6db6c59f8783a742af65e9b30c7687408d5983ba; YD00517437729195:WM_NI=lEQ/IgzDntCyopWZtBMEXuAZ/ubtjc6xPlHTP3n8QHW1s1w6bXJ4HCNdSGOjEije1A1/ffYtzuJq1d5W/BebF2jmjqtwbYuF8SBAQ9kpAQVqXwRtgB3yzRQJsLkV3uwbcFM=; YD00517437729195:WM_NIKE=9ca17ae2e6ffcda170e2e6ee8fef33b8889a8dfc6ba3b08ba2d84b969e8b87d15f9aa699a3bc3cbb8c9e97fb2af0fea7c3b92ab09b9fd0cc6dba9ca295c273adb099bbf74a8f9b9896c452edabae95e969f6a8a08dcb6094eda2d9d44495efbdd2f63ea28886d5cb498ebc9da8ce6287b7a686b44bf3f1aa87dc80baab8692b470f789e1a3e86a86eea68ee97bb8a9aebaf84efc8d8ea2d433a9b08fb2d57b9ca99e95e64af1a7b6b3d442b194a5b7b27fb5beada7d037e2a3; gdxidpyhxdE=OI7WZLerCjMqftxEXq0ZvDo/gBqyVwAEND9x1+AHhe1pe\D6NVsPoj1VwQHW7caVfBS\hwwBpTEMJ0cNyL9c1A4Cp4A2sYn5Xi3c+GjN5ZLTJhAxzd3rWIjZG5fVWnki//kOS2K+l2\jDJlBY1n9U9Bmvv+raqJmKZZqTL6uTBsZ+5ZS:1652105416421; z_c0="2|1:0|10:1652104528|4:z_c0|92:Mi4xSkN0bEJnQUFBQUFBOEotbFNJNk5FeWNBQUFDRUFsVk5UNnFnWWdEUlZINzZqYVFuVGlhZDFadTRBWWNCd0tTdHdR|dfd4846cca7c46cc4b512ef3574051350571ae18ea20ad1815efe71ac3c45cc2"; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1651671713,1651908657,1652104513,1652183970; NOT_UNREGISTER_WAITING=1; tst=r; Hm_lpvt_98beee57fd2ef70ccdd5ca52b9740c49=1652184559; SESSIONID=SKJvMgXr1BjPmBQqrqXU1CwJqjozChEYVKEJGWF9lX9; JOID=U1AWBUkkTyfJsNa9ECPltbM3ZAUBfg9ou_647mcWBEac2ZflassUXKy61LMRBswkiVZzO4UXG_x6J3EnFjtn0WA=; osd=VFoUA0ojRSXPs9G3EiXmsrk1YgYGdA1uuPmy7GEVA0ye35TiYMkSX6uw1rUSAcYmj1V0MYcRGPtwJXckETFl12M=; KLBRSID=ed2ad9934af8a1f80db52dcb08d13344|1652184558|1652183964',
    'Host':'www.zhihhu.com',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36',
}
r = requests.get("https://www.zhihu.com/",headers=headers)
print(r.text)

In [17]:
import requests

cookies = '_zap=914c7061-bc67-40d8-9d2f-307d9203368a; d_c0="APCfpUiOjROPTny1k41UpIDGLfgRghMrJsg=|1628697163"; _xsrf=ZFeEdqQ2pLrCK55wCH5EZVkKZkOkpXRf; __snaker__id=jxZ3erivx0VO75TL; _9755xjdesxxd_=32; YD00517437729195:WM_TID=eHSkDGk8jLlAQUFAQAJ79YQGeWeQef1b; q_c1=45705d6745e14ca2aa7e8db26508061c|1645972962000|1645972962000; captcha_session_v2=2|1:0|10:1652104512|18:captcha_session_v2|88:d2xKZUROR3pGb01nTEQrT2gvNytrR2k1Uk5MYVo2aDVlUUUzeFVESjFlazBkUEVwa3RrYlJqaS85TkJYRHFjNw==|5741ff6f88b2c3bd13c26dce6db6c59f8783a742af65e9b30c7687408d5983ba; YD00517437729195:WM_NI=lEQ/IgzDntCyopWZtBMEXuAZ/ubtjc6xPlHTP3n8QHW1s1w6bXJ4HCNdSGOjEije1A1/ffYtzuJq1d5W/BebF2jmjqtwbYuF8SBAQ9kpAQVqXwRtgB3yzRQJsLkV3uwbcFM=; YD00517437729195:WM_NIKE=9ca17ae2e6ffcda170e2e6ee8fef33b8889a8dfc6ba3b08ba2d84b969e8b87d15f9aa699a3bc3cbb8c9e97fb2af0fea7c3b92ab09b9fd0cc6dba9ca295c273adb099bbf74a8f9b9896c452edabae95e969f6a8a08dcb6094eda2d9d44495efbdd2f63ea28886d5cb498ebc9da8ce6287b7a686b44bf3f1aa87dc80baab8692b470f789e1a3e86a86eea68ee97bb8a9aebaf84efc8d8ea2d433a9b08fb2d57b9ca99e95e64af1a7b6b3d442b194a5b7b27fb5beada7d037e2a3; gdxidpyhxdE=OI7WZLerCjMqftxEXq0ZvDo/gBqyVwAEND9x1+AHhe1pe\D6NVsPoj1VwQHW7caVfBS\hwwBpTEMJ0cNyL9c1A4Cp4A2sYn5Xi3c+GjN5ZLTJhAxzd3rWIjZG5fVWnki//kOS2K+l2\jDJlBY1n9U9Bmvv+raqJmKZZqTL6uTBsZ+5ZS:1652105416421; z_c0="2|1:0|10:1652104528|4:z_c0|92:Mi4xSkN0bEJnQUFBQUFBOEotbFNJNk5FeWNBQUFDRUFsVk5UNnFnWWdEUlZINzZqYVFuVGlhZDFadTRBWWNCd0tTdHdR|dfd4846cca7c46cc4b512ef3574051350571ae18ea20ad1815efe71ac3c45cc2"; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1651671713,1651908657,1652104513,1652183970; NOT_UNREGISTER_WAITING=1; tst=r; Hm_lpvt_98beee57fd2ef70ccdd5ca52b9740c49=1652184559; SESSIONID=SKJvMgXr1BjPmBQqrqXU1CwJqjozChEYVKEJGWF9lX9; JOID=U1AWBUkkTyfJsNa9ECPltbM3ZAUBfg9ou_647mcWBEac2ZflassUXKy61LMRBswkiVZzO4UXG_x6J3EnFjtn0WA=; osd=VFoUA0ojRSXPs9G3EiXmsrk1YgYGdA1uuPmy7GEVA0ye35TiYMkSX6uw1rUSAcYmj1V0MYcRGPtwJXckETFl12M=; KLBRSID=ed2ad9934af8a1f80db52dcb08d13344|1652184558|1652183964'
jar = requests.cookies.RequestsCookieJar()
headers = {
    'Host':'www.zhihu.com',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'
}
for cookie in cookies.split(';'):
    key,value = cookie.split('=',1)
    jar.set(key,value)
r = requests.get("http://www.zhihu.com",cookies=jar,headers=headers)
print(r.text)

<!doctype html>
<html lang="zh" data-hairline="true" data-theme="light"><head><meta charSet="utf-8"/><title data-rh="true">首页 - 知乎</title><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1"/><meta name="renderer" content="webkit"/><meta name="force-rendering" content="webkit"/><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><meta name="google-site-verification" content="FTeR0c8arOPKh8c5DYh_9uu98_zJbaWw53J-Sch9MTg"/><meta name="description" property="og:description" content="知乎，中文互联网高质量的问答社区和创作者聚集的原创内容平台，于 2011 年 1 月正式上线，以「让人们更好的分享知识、经验和见解，找到自己的解答」为品牌使命。知乎凭借认真、专业、友善的社区氛围、独特的产品机制以及结构化和易获得的优质内容，聚集了中文互联网科技、商业、影视、时尚、文化等领域最具创造力的人群，已成为综合性、全品类、在诸多领域具有关键影响力的知识分享社区和创作者聚集的原创内容平台，建立起了以社区驱动的内容变现商业模式。"/><link data-rh="true" rel="apple-touch-icon" href="https://static.zhihu.com/heifetz/assets/apple-touch-icon-152.a53ae37b.png"/><link data-rh="true" rel="apple-touch-icon" href="https://static.zhihu.com/heifetz/assets/apple-touch-icon-152.a53ae37b.png" size

#### 会话维持

在requests中，如果直接利用get()或者post()等方法可以做到网页的请求，但这实际上是不同的会话，也就是说相当于用浏览器打开了不同的页面。

解决上述打开了两个浏览器的页面问题，我们可以维持同一个会话，也就是相当于打开了一个新的浏览器选项卡而不是新开一个浏览器。这时候就有了Session对象。

In [19]:
import requests

requests.get('http://httpbin.org/cookies/set/number/123456789')
r = requests.get('http://httpbin.org/cookies')
print(r.text)

{
  "cookies": {}
}



并不能获取cookies。  
再使用Session对象

In [20]:
import requests

s = requests.Session()
s.get('http://httpbin.org/cookies/set/number/123456789')
r = s.get('http://httpbin.org/cookies')
print(r.text)

{
  "cookies": {
    "number": "123456789"
  }
}



#### SSL证书验证

此外，requests还提供了证书验证的功能。当发送HTTP请求的时候，它会检查SSL证书，我们可以使用verify参数控制是否检查此证书。如果不加verify参数的话，默认是True，会自动验证。

In [24]:
import requests

response = requests.get('https://www.12306.cn',verify=False)
print(response.status_code)

200


D:\Anacoda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.12306.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\Anacoda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.12306.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


#### 代理设置

对于某些网站，在测试的时候请求几次，能正常获取内容。但是一旦开始大规模爬取，对于大规模且频繁的请求，网站可能会弹出验证码，或者跳转到登陆认证页面，更甚者可以能会直接封禁客户端的IP，导致一定时段内无法访问。

In [25]:
import requests

proxies = {
    "http":"http://10.10.1.10:3128",
    "http":"http://10.10.1.10:1080",
}

requests.get("https://www.taobao.com",proxies=proxies)

<Response [200]>

代理如上使用，可以换成自己的有效代理

#### 身份认证

In [27]:
import requests
from requests.auth import HTTPBasicAuth

r = requests.get('http://localhost:5000',auth=HTTPBasicAuth('username','password'))
print(r.status_code)

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000241955E2CD0>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))

## 正则表达式

|Model|Description|
|---|----|
|\w|匹配字母数字及下划线|
|\W|匹配不是字母、数字及下划线的字符|
|\s|匹配任意空白字符、等价于[\t\n\r\f]|
|\S|

### match()方法

In [28]:
import re

content = 'Hello 123 4567 World_This is a Regex Demo'
print(len(content))
result = re.match('^Hello\s\d\d\d\s\d{4}\s\w{10}',content)
print(result)
print(result.group())
print(result.span())

41
<re.Match object; span=(0, 25), match='Hello 123 4567 World_This'>
Hello 123 4567 World_This
(0, 25)


用它来匹配这个长字符串。开头的^是匹配字符串的开头，也就是以Hello开头；然后\s匹配空白字符，用来匹配目标字符串的空格；\d匹配数字，3个\d匹配123；然后再写一个\s匹配空格；后面还有4567，跟{4}以代表匹配前面的规划4次，也就是匹配4个数字；然后再紧接着1个空白字符，最后\w{10}匹配10个字母及下划线。

可以看到结果是SRE_Match,这证明匹配成功。该对象有两个方法，group()方法可以输出匹配到的内容，结果是Hello 123 4567 World——This,这恰好是正则表式所匹配的内容；span（）方法可以输出匹配的范围，结果是（0，25），这就是匹配到的结果字符串在原字符串的位置范围。

#### 匹配目标

可以使用（）将想提取出来的字符串括起来。（）实际上标记了一个子表达式的开始和结束标志，被标记的每个子表达式会依次对应每一个分组，调用group()方法传入分组即可获取提取的结果，

In [29]:
import re

content = 'Hello 1234567 World_This is a Regex Demo'
result = re.match('^Hello\s(\d+)\sWorld',content)
print(result)
print(result.group())
print(result.group(1))
print(result.span())

<re.Match object; span=(0, 19), match='Hello 1234567 World'>
Hello 1234567 World
1234567
(0, 19)


可以看到，我们成功得到了1234567，这里用的是group(1)，它与group()有所不同，后者会输出完整的匹配结果，而前者会输出第一个被（）包围的结果。

#### 通用匹配

刚才我们写的正则表达式其实比较复杂，出现空白字符我们就写\s匹配，出先数字用\d。工作量大  
使用.* 其中.可以匹配任意字符（除换行符），* 代表匹配前面的字符无限次，所以它们组合在一起就可以匹配任意字符了。

In [32]:
import re

content = 'Hello 123 4567 World_This is a Regex Demo'
result = re.match('^Hello.*Demo$',content)
print(result)
print(result.group())
print(result.span())

<re.Match object; span=(0, 41), match='Hello 123 4567 World_This is a Regex Demo'>
Hello 123 4567 World_This is a Regex Demo
(0, 41)


#### 贪婪

In [33]:
import re

content = 'Hello 1234567 World_This is a Regex Demo'
result = re.match('^He.*(\d+).*Demo$',content)
print(result)
print(result.group(1))

<re.Match object; span=(0, 40), match='Hello 1234567 World_This is a Regex Demo'>
7


在贪婪的匹配下，.* 会匹配尽可能多的字符。正则表达式中 .* 后面是\d+，也就是少一个数字，并没有指定具体多数字，因此，.* 就尽可能匹配多的字符，这里就把123456匹配了，给\d+留下一个可满足条件的数字7，最后得到的内容只有数字7了。  
只需要使用非贪婪匹配就好了 .*?

In [34]:
import re

content = 'Hello 1234567 World_This is a Regex Demo'
result = re.match('^He.*?(\d+).*Demo$',content)
print(result)
print(result.group(1))

<re.Match object; span=(0, 40), match='Hello 1234567 World_This is a Regex Demo'>
1234567


非贪婪匹配就是尽可能匹配少的字符。当 .* ? 匹配到Hello后面的空白字符时，再往后的字符就是数字了。而\d+恰好可以匹配，这里 .*?就不再匹配

In [35]:
import re

content = 'http://weibo.com/comment/kEraCN'
result1 = re.match('http.*?comment/(.*?)',content)
result2 = re.match('http.*?comment/(.*)',content)
print('result1',result1.group(1))
print('result2',result2.group(1))

result1 
result2 kEraCN


.* ?处在字符串的结尾就有可能匹配不到任何内容了，因为其会尽可能少的匹配字符

#### 修饰符

In [37]:
import re

content = """Hello 1234567 World_This
is a Regex Demo
"""
result = re.match('^He.*?(\d+).*?Demo$',content)
print(result.group(1))

AttributeError: 'NoneType' object has no attribute 'group'

运行直接报错，也就是说正则表达式没有匹配到这个字符串，返回结果为None。  
.* ? 无法匹配换行符，所以导致匹配失败

In [39]:
result = re.match('^He.*?(\d+).*?Demo$',content,re.S)
print(result.group(1))

1234567


re.S这个修饰符是使.匹配包括换行符在内的所有字符。在网页中经常使用，因为HTML节点经常会有换行，加上它，就可以匹配节点与节点之间的换行了

|修饰符|描述|
|---|---|
|re.I|使匹配对大小写不敏感|
|re.L|做本地化识别（locate-aware）匹配|
|re.M|多行匹配，影响^和$|
|re.S|使.匹配包括换行在内的所有字符|
|re.U|使用Unicode字符集解析字符。影响\w,\W,\b和\B|
|re.X|该标志通过给予你更灵活的格式以便你将正则表达式写得更易于理解|

#### 转义匹配

In [41]:
import re

content = '(百度)www.baidu.com'
result = re.match('\(百度\)www\.baidu\.com',content)
print(result)

<re.Match object; span=(0, 17), match='(百度)www.baidu.com'>


对于用于正则匹配模式的特殊字符串时，在前面加反斜杠转义一下即可。

#### search()

In [42]:
import re

content = 'Extra stings Hello 1234567 World_This is a Regex Demo Extra stings'
result = re.match('Hello.*?(\d+).*?Demo',content)
print(result)

None


match()方法在使用时需要考虑到开头的内容，这在做匹配时并不方便。它更适合用来检测某个字符串是否符合某个正则表达式的规则  
有另外一个search()方法，它在匹配时会扫描整个字符串，然后返回第一个成功匹配的结果。在匹配时search（）方法会依次扫描字符串，直到找到第一个符合规则的字符串，然后返回匹配内容。

In [45]:
import re

content = 'Extra stings Hello 1234567 World_This is a Regex Demo Extra stings'
result = re.search('Hello.*?(\d+).*?Demo',content)
print(result)
print(result.group(1))

<re.Match object; span=(13, 53), match='Hello 1234567 World_This is a Regex Demo'>
1234567


In [46]:
html = """<div id="song-list">
<h2 class="title">经典老歌</h2>
<p class="introduction">
经典老歌列表
</p>
<ul id="list" class="list-group">
<li data-view="2">一路有你</ul>
<li data-view="7">
<a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
</li>
<li data-view="4" class="active">
<a href="/3.mp3" singer="齐秦">往事随风</a>
</li>
<li data-view="6"><a href="/4.mp3" singer="beyond">光辉岁月</a></li>
<li data-view="5"><a href="/5.mp3" singer="陈慧琳">记事本</a></li>
<li data-view="5">
<a href="/6.mp3" singer="邓丽君">但愿人长久</a>
</li>
</ul>
</div>
"""

尝试提取class为active的li节点内部的超链接包含的歌手名和歌名，此时需要提取第三个li节点下的singer属性和文本

以li开头，然后寻找一个标识符active，中间的部分可以用.* ?来匹配。接着，要提取singer这个属性值，所以还需要写入singer="(.* ?)"，这里需要提取的部分用小括号起来，以便用group()方法提取出来，它的两侧边界时双引号。然后还需要匹配a节点的文本，其中它的左边界是<，右边界是</a>。然后目标内容依然用(.* ?)来匹配，所以最后的正则表达式变为：

In [47]:
result = re.search('<li.*?active.*?singer="(.*?)">(.*?)</a>',html,re.S)
if result:
    print(result.group(1),result.group(2))

齐秦 往事随风


若删掉active,search（）方法会返回第一个符合条件的匹配目标，这里结果就变了

In [48]:
result = re.search('<li.*?singer="(.*?)">(.*?)</a>',html,re.S)
if result:
    print(result.group(1),result.group(2))

任贤齐 沧海一声笑


若去掉re.S,则结果变为

In [49]:
result = re.search('<li.*?singer="(.*?)">(.*?)</a>',html)
if result:
    print(result.group(1),result.group(2))

beyond 光辉岁月


#### findall()

前面介绍的search()方法的用法，可以返回匹配的正则表达式的第一个内容，但是如过匹配所有正则表达式的内容，则需要使用findall()方法。

In [53]:
results = re.findall('<li.*?href="(.*?)".*?singer="(.*?)">(.*?)</a>',html,re.S)
print(results)
print(type(result))
for result in results:
    print(result)
    print(result[0],result[1],result[2])

[('/2.mp3', '任贤齐', '沧海一声笑'), ('/3.mp3', '齐秦', '往事随风'), ('/4.mp3', 'beyond', '光辉岁月'), ('/5.mp3', '陈慧琳', '记事本'), ('/6.mp3', '邓丽君', '但愿人长久')]
<class 'tuple'>
('/2.mp3', '任贤齐', '沧海一声笑')
/2.mp3 任贤齐 沧海一声笑
('/3.mp3', '齐秦', '往事随风')
/3.mp3 齐秦 往事随风
('/4.mp3', 'beyond', '光辉岁月')
/4.mp3 beyond 光辉岁月
('/5.mp3', '陈慧琳', '记事本')
/5.mp3 陈慧琳 记事本
('/6.mp3', '邓丽君', '但愿人长久')
/6.mp3 邓丽君 但愿人长久


#### sub()

除了使用正则表达式提取信息外，有时候还需要借助它来修改文本。

In [55]:
content = '54aK1324WDiawjdda113sdADW13a'
content = re.sub('\d+','',content)
print(content)

aKWDiawjddasdADWa


In [56]:
results = re.findall('<li.*?>\s*?(<a.*?>)?(\w+)(</a>)?\s*?</li>',html,re.S)
for result in results:
    print(result[1])

沧海一声笑
往事随风
光辉岁月
记事本
但愿人长久


此时借助sub()方法就比较简单了。可以先用sub()方法将a节点去掉，只留下文本然后再利用findall()方法提取就可以了

In [58]:
html = re.sub('<a.*?>|</a>','',html)
print(html)
results = re.findall('<li.*?>(.*?/li>)',html,re.S)
for result in results:
    print(result.strip())

<div id="song-list">
<h2 class="title">经典老歌</h2>
<p class="introduction">
经典老歌列表
</p>
<ul id="list" class="list-group">
<li data-view="2">一路有你</ul>
<li data-view="7">
沧海一声笑
</li>
<li data-view="4" class="active">
往事随风
</li>
<li data-view="6">光辉岁月</li>
<li data-view="5">记事本</li>
<li data-view="5">
但愿人长久
</li>
</ul>
</div>

一路有你</ul>
<li data-view="7">
沧海一声笑
</li>
往事随风
</li>
光辉岁月</li>
记事本</li>
但愿人长久
</li>


#### compile()方法

可以将正则字符串编译成正则表达式对象，一遍后面再匹配中复用

In [59]:
import re 

content1 = '2022-05-11 12:00'
content2 = '2022-05-11 12:55'
content3 = '2022-05-11 13:55'
pattern = re.compile('\d{2}:\d{2}')
result1 = re.sub(pattern,'',content1)
result2 = re.sub(pattern,'',content2)
result3 = re.sub(pattern,'',content3)
print(result1,result2,result3)

2022-05-11  2022-05-11  2022-05-11 
